#### Field customization

Optionally, the `Field` function can be used to provide extra information about the field and validations. It has the following arguments.

* `default`: (a positional argument) the default value of the field. Since the `Field` replaces the field's default, this first argument can be used to set the default. Use ellipsis (`...`) to indicate the field is required.

* `default_factory`: a zero-argument callable that will be called when a default value is needed for this field. Among other purposes, this can be used to set dynamic default values. It is forbidden to set both `default` and `default_factory`.

* `alias`: the public name of the field.

* `title`: if omitted, `field_name.title()` is used.

* `description`: if omitted and the annotation is a sub-model, the docstring of the sub-model will be used.

* `exclude`: exclude this field when dumping (`.dict` and `.json`) the instance. The exact syntax and configuration options are described in details in the `exporting models section`.

* `include`: include (only) this field when dumping (`.dict` and `.json`) the instance. The exact syntax and configuration options are described in details in the `exporting models section`.

* `const`: this argument must be the same as the field's default value if present.

* `gt`: for numeric values (`int`, `float`, `Decimal`), adds a validation of "greater than" and an annotation of `exclusiveMinimum` to the JSON Schema.

* `ge`: for numeric values, this adds a validation of "greater than or equal" and an annotation of `minimum` to the JSON Schema.

* `lt`: for numeric values, this adds a validation of "less than" and an annotation of `exclusiveMaximum` to the JSON Schema.

* `le`: for numeric values, this adds a validation of "less than or equal" and an annotation of `maximum` to the JSON Schema.

* `multiple_of`: for numeric values, this adds a validation of "a multiple of" and an annotation of multipleOf to the JSON Schema.

* `max_digits`: for `Decimal` values, this adds a validation to have a maximum number of digits within the decimal. It does not include a zero before the decimal point or trailing decimal zeroes.

* `decimal_places`: for `Decimal` values, this adds a validation to have at most a number of decimal places allowed. It does not include trailing decimal zeroes.

* `min_items`: for list values, this adds a corresponding validation and an annotation of `minItems` to the JSON Schema.

* `max_items`: for list values, this adds a corresponding validation and an annotation of `maxItems` to the JSON Schema.

* `unique_items`: for list values, this adds a corresponding validation and an annotation of `uniqueItems` to the JSON Schema.

* `min_length`: for string values, this adds a corresponding validation and an annotation of `minLength` to the JSON Schema.

* `max_length`: for string values, this adds a corresponding validation and an annotation of `maxLength` to the JSON Schema.

* `allow_mutation`: a boolean which defaults to `True`. When False, the field raises a `TypeError` if the field is assigned on an instance. The model config must set `validate_assignment` to `True` for this check to be performed.

* `regex`: for string values, this adds a Regular Expression validation generated from the passed string and an annotation of `pattern` to the JSON Schema.

> _pydantic_ validates strings using `re.match`, which treats regular expressions as implicitly anchored at the beginning. On the contrary, JSON Schema validators treat the `pattern` keyword as implicitly unanchored, more like what `re.search` does.
>
> For interoperability, depending on your desired behavior, either explicitly anchor your regular expressions with `^` (e.g. `^foo` to match any string starting with `foo`), or explicitly allow an arbitrary prefix with `.*?` (e.g. `.*?foo` to match any string containing the substring `foo`).

* `repr`: a boolean which defaults to `True`. When False, the field shall be hidden from the object representation.

* `**` any other keyword arguments (e.g. examples) will be added verbatim to the field's schema.

Instead of using `Field`, the `fields` property of `the Config class` can be used to set all of the arguments above except `default`.

##### Unenforced Field constraints

If _pydantic_ finds constraints which are not being enforced, an error will be raised. If you want to force the constraint to appear in the schema, even though it's not being checked upon parsing, you can use variadic arguments to `Field()` with the raw schema attribute name.

In [1]:
from uuid import uuid4
from typing import Annotated
from pydantic import BaseModel, Field, PositiveInt

In [2]:
try:
    # this won't work since PositiveInt takes precedence over the
    # constraints defined in Field meaning they're ignored
    class Model(BaseModel):
        foo: PositiveInt = Field(..., lt=10)
except ValueError as e:
    print(e)

On field "foo" the following field constraints are set but not enforced: lt. 
For more details see https://pydantic-docs.helpmanual.io/usage/schema/#unenforced-field-constraints


In [3]:
# but you can set the schema attribute directly:
# (Note: here exclusiveMaximum will not be enforce)
class Model(BaseModel):
    foo: PositiveInt = Field(..., exclusiveMaximum=10)

In [4]:
print(Model.schema())

{'title': 'Model', 'type': 'object', 'properties': {'foo': {'title': 'Foo', 'exclusiveMaximum': 10, 'exclusiveMinimum': 0, 'type': 'integer'}}, 'required': ['foo']}


In [5]:
print(Model.schema_json(indent=4))

{
    "title": "Model",
    "type": "object",
    "properties": {
        "foo": {
            "title": "Foo",
            "exclusiveMaximum": 10,
            "exclusiveMinimum": 0,
            "type": "integer"
        }
    },
    "required": [
        "foo"
    ]
}


In [6]:
# if you find yourself needing this, an alternative is to declare
# the constraints in Field (or you could use conint())
# here both constraints will be enforced:
class Model(BaseModel):
    # Here both constraints will be applied and the schema
    # will be generated correctly
    foo: int = Field(..., gt=0, lt=10)

In [7]:
print(Model.schema())

{'title': 'Model', 'type': 'object', 'properties': {'foo': {'title': 'Foo', 'exclusiveMinimum': 0, 'exclusiveMaximum': 10, 'type': 'integer'}}, 'required': ['foo']}


In [8]:
print(Model.schema_json(indent=4))

{
    "title": "Model",
    "type": "object",
    "properties": {
        "foo": {
            "title": "Foo",
            "exclusiveMinimum": 0,
            "exclusiveMaximum": 10,
            "type": "integer"
        }
    },
    "required": [
        "foo"
    ]
}


##### typing.Annotated Fields

Rather than assigning a `Field` value, it can be specified in the type hint with `typing.Annotated`.

In [9]:
class Foo(BaseModel):
    id: Annotated[str, Field(default_factory=lambda: uuid4().hex)]
    name: Annotated[str, Field(max_length=256)] = "Bar"

`Field` can only be supplied once per field - an error will be raised if used in `Annotated` and as the assigned value. Defaults can be set outside `Annotated` as the assigned value or with `Field.default_factory` inside `Annotated` - the `Field.default` argument is not supported inside `Annotated`.

For versions of Python prior to 3.9, `typing_extensions.Annotated` can be used.